# Evolution Metrics Visualization

This notebook visualizes the detailed metrics collected during evolutionary runs.

## Setup

First, run an evolution with metrics export:
```bash
python prototype.py --generations 5 --population 10 --duration 0.1 --export-metrics metrics/run.json
```

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Load Metrics Data

In [ ]:
# Load the metrics file
metrics_file = '../metrics/test_run.json'  # Change this to your metrics file

with open(metrics_file) as f:
    data = json.load(f)

print(f"Target Number: {data['target_number']}")
print(f"Generations: {data['generation_count']}")
print(f"Population Size: {data['population_size']}")
print(f"Evaluation Duration: {data['evaluation_duration']}s")

## Fitness Over Generations

In [ ]:
# Extract fitness data
generations = range(data['generation_count'])
fitness_by_gen = [
    [metrics['candidate_count'] for metrics in generation]
    for generation in data['metrics_history']
]

max_fitness = [max(gen) for gen in fitness_by_gen]
avg_fitness = [np.mean(gen) for gen in fitness_by_gen]
min_fitness = [min(gen) for gen in fitness_by_gen]

# Plot
plt.figure(figsize=(14, 7))
plt.plot(generations, max_fitness, 'b-', label='Best Fitness', linewidth=2.5, marker='o')
plt.plot(generations, avg_fitness, 'g--', label='Average Fitness', linewidth=2, marker='s')
plt.fill_between(generations, min_fitness, max_fitness, alpha=0.2)

plt.xlabel('Generation', fontsize=12)
plt.ylabel('Fitness (candidates found)', fontsize=12)
plt.title('Evolutionary Progress: Fitness Over Generations', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nFitness improvement: {max_fitness[0]} → {max_fitness[-1]} "+
      f"({((max_fitness[-1]/max(max_fitness[0], 1) - 1) * 100):.1f}% change)")

## Timing Breakdown Analysis

In [ ]:
# Calculate average timing percentages across all generations
timing_categories = ['candidate_generation', 'modulus_filtering', 'smoothness_check']

avg_timings = {cat: [] for cat in timing_categories}

for generation in data['metrics_history']:
    gen_timings = {cat: [] for cat in timing_categories}
    
    for metrics in generation:
        total_time = sum(metrics['timing_breakdown'].values())
        if total_time > 0:
            for cat in timing_categories:
                pct = (metrics['timing_breakdown'][cat] / total_time) * 100
                gen_timings[cat].append(pct)
    
    for cat in timing_categories:
        avg_timings[cat].append(np.mean(gen_timings[cat]) if gen_timings[cat] else 0)

# Plot stacked area chart
plt.figure(figsize=(14, 7))
plt.stackplot(generations, 
              avg_timings['candidate_generation'],
              avg_timings['modulus_filtering'],
              avg_timings['smoothness_check'],
              labels=['Candidate Generation', 'Modulus Filtering', 'Smoothness Check'],
              alpha=0.8)

plt.xlabel('Generation', fontsize=12)
plt.ylabel('Time Allocation (%)', fontsize=12)
plt.title('Time Allocation Across Evaluation Phases', fontsize=14, fontweight='bold')
plt.legend(loc='upper left', fontsize=11)
plt.grid(True, alpha=0.3)
plt.ylim(0, 100)
plt.tight_layout()
plt.show()

## Rejection Statistics

In [ ]:
# Analyze rejection patterns
rejection_data = {
    'modulus_filter': [],
    'min_hits': [],
    'passed': []
}

for generation in data['metrics_history']:
    gen_rejections = {'modulus_filter': 0, 'min_hits': 0, 'passed': 0}
    
    for metrics in generation:
        for key in rejection_data.keys():
            gen_rejections[key] += metrics['rejection_stats'][key]
    
    total = sum(gen_rejections.values())
    if total > 0:
        for key in rejection_data.keys():
            rejection_data[key].append((gen_rejections[key] / total) * 100)

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Stacked bar chart
x = np.arange(len(generations))
width = 0.6

ax1.bar(x, rejection_data['modulus_filter'], width, label='Modulus Filter Rejection', alpha=0.8)
ax1.bar(x, rejection_data['min_hits'], width, bottom=rejection_data['modulus_filter'],
        label='Min Hits Rejection', alpha=0.8)
ax1.bar(x, rejection_data['passed'], width, 
        bottom=np.array(rejection_data['modulus_filter']) + np.array(rejection_data['min_hits']),
        label='Passed', alpha=0.8, color='green')

ax1.set_xlabel('Generation', fontsize=11)
ax1.set_ylabel('Percentage (%)', fontsize=11)
ax1.set_title('Rejection vs Acceptance Rates', fontsize=13, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(generations)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3, axis='y')

# Line plot for passed percentage
ax2.plot(generations, rejection_data['passed'], 'g-', marker='o', linewidth=2.5, markersize=8)
ax2.fill_between(generations, rejection_data['passed'], alpha=0.3, color='green')
ax2.set_xlabel('Generation', fontsize=11)
ax2.set_ylabel('Pass Rate (%)', fontsize=11)
ax2.set_title('Evolution of Pass Rate', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nPass rate improvement: {rejection_data['passed'][0]:.2f}% → {rejection_data['passed'][-1]:.2f}%")

## Smoothness Quality Analysis

In [ ]:
# Analyze smoothness scores (lower = smoother)
avg_smoothness_by_gen = []

for generation in data['metrics_history']:
    all_scores = []
    for metrics in generation:
        if metrics['smoothness_scores']:
            # Filter out inf values
            valid_scores = [s for s in metrics['smoothness_scores'] if np.isfinite(s)]
            all_scores.extend(valid_scores)
    
    if all_scores:
        # Use log scale for better visualization
        avg_smoothness_by_gen.append(np.log10(np.mean(all_scores)))
    else:
        avg_smoothness_by_gen.append(None)

# Plot
plt.figure(figsize=(14, 7))
valid_gens = [g for g, s in zip(generations, avg_smoothness_by_gen) if s is not None]
valid_scores = [s for s in avg_smoothness_by_gen if s is not None]

plt.plot(valid_gens, valid_scores, 'r-', marker='o', linewidth=2.5, markersize=8)
plt.xlabel('Generation', fontsize=12)
plt.ylabel('Log10(Average Smoothness Ratio)', fontsize=12)
plt.title('Smoothness Quality Evolution (Lower = Better)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

if len(valid_scores) >= 2:
    print(f"\nSmoothness trend: {'Improving (↓)' if valid_scores[-1] < valid_scores[0] else 'Degrading (↑)'}")

## Best Strategy Analysis

In [ ]:
# Find best strategy from last generation
last_gen = data['metrics_history'][-1]
best_idx = max(range(len(last_gen)), key=lambda i: last_gen[i]['candidate_count'])
best_metrics = last_gen[best_idx]

print("=" * 60)
print("BEST STRATEGY FROM FINAL GENERATION")
print("=" * 60)
print(f"\nFitness: {best_metrics['candidate_count']} candidates found")
print(f"\nTiming Breakdown:")
total_time = sum(best_metrics['timing_breakdown'].values())
for phase, time_val in best_metrics['timing_breakdown'].items():
    pct = (time_val / total_time * 100) if total_time > 0 else 0
    print(f"  {phase.replace('_', ' ').title()}: {pct:.1f}%")

print(f"\nRejection Stats:")
total_attempts = sum(best_metrics['rejection_stats'].values())
for stat, count in best_metrics['rejection_stats'].items():
    pct = (count / total_attempts * 100) if total_attempts > 0 else 0
    print(f"  {stat.replace('_', ' ').title()}: {count:,} ({pct:.1f}%)")

if best_metrics['smoothness_scores']:
    valid_scores = [s for s in best_metrics['smoothness_scores'] if np.isfinite(s)]
    if valid_scores:
        print(f"\nSmoothness Quality:")
        print(f"  Average: {np.mean(valid_scores):.2e}")
        print(f"  Min: {np.min(valid_scores):.2e}")
        print(f"  Max: {np.max(valid_scores):.2e}")

if best_metrics['example_candidates']:
    print(f"\nExample Smooth Candidates:")
    for i, candidate in enumerate(best_metrics['example_candidates'][:3], 1):
        print(f"  {i}. {candidate}")

## Summary Statistics

In [ ]:
print("=" * 60)
print("EVOLUTION SUMMARY")
print("=" * 60)

total_candidates_found = sum(
    metrics['candidate_count']
    for generation in data['metrics_history']
    for metrics in generation
)

total_evaluations = data['generation_count'] * data['population_size']
total_time = total_evaluations * data['evaluation_duration']

print(f"\nTotal Evaluations: {total_evaluations}")
print(f"Total Candidates Found: {total_candidates_found:,}")
print(f"Average per Evaluation: {total_candidates_found / total_evaluations:.1f}")
print(f"Total Evaluation Time: {total_time:.1f}s")
print(f"Candidates per Second: {total_candidates_found / total_time:.1f}")

improvement = ((max_fitness[-1] / max(max_fitness[0], 1)) - 1) * 100
print(f"\nBest Fitness Improvement: {improvement:+.1f}%")
print(f"Final Best Strategy: {max_fitness[-1]:,} candidates")